In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

!mkdir /content/data
%cd /content/data
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip 

Mounted at /content/gdrive
/content/data
--2021-10-06 19:20:06--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  26.1MB/s    in 17s     

2021-10-06 19:20:23 (13.6 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
torch.manual_seed(2)
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision.models as models
!pip install torchinfo
!pip install torchattacks
from torchinfo import summary
from torchvision.models.resnet import _resnet,BasicBlock

import torchvision.utils

     |████████████████████████████████| 100 kB 5.9 MB/s 


In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
transform = transforms.Compose([ 
    transforms.Resize((224)), 
    transforms.ToTensor(),
    normalize
])

dataset = torchvision.datasets.ImageFolder('/content/data/tiny-imagenet-200/train', transform=transform)
#split the data
train_data, val_data, test_data = torch.utils.data.random_split(dataset, [80000, 10000, 10000], generator=torch.Generator().manual_seed(42))  ##set seed to ensure consistency
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=2, sampler = SubsetRandomSampler(range(80000)))
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False,num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform1)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

valset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform2)
val_loader = torch.utils.data.DataLoader(
    valset, batch_size=100, shuffle=False, num_workers=2, sampler = SubsetRandomSampler(range(5000)))

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform2)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2, sampler = SubsetRandomSampler(range(5000,10000)))

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
from torch.nn import Conv2d,AvgPool2d,Linear,Sequential,Dropout,BatchNorm2d,ModuleList,BatchNorm1d
import torch.nn.functional as F
import numpy as np
import math
from torch.autograd import Variable

## https://blog.paperspace.com/attention-mechanisms-in-computer-vision-cbam/
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = torch.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out
        return x

class Base(nn.Module):
    def freeze(self):
        for param in self.base_model.parameters():
                param.requires_grad = False
    
    def unfreeze(self):
        for param in self.base_model.parameters():
                param.requires_grad = True
    
    def attach_fea_out(self,classname,input,output):
        self.features.append(output)

    def attach_fea_in(self,classname,input,output):
        self.features.append(input[0])

    def __init__(self,trainable = True,attention=False):
        super(Base,self).__init__()
        self.features = []
        self.channel_size = []
        
        self.base_model = models.resnet18(pretrained=False)
        used_blocks = ['layer1', 'layer2','layer3','layer4']
        unused_blocks = ['avgpool','fc']

        for block in used_blocks:
            getattr(self.base_model,block).register_forward_hook(self.attach_fea_out)

        for block in unused_blocks:
             setattr(self.base_model,block,nn.Identity())
        
        if not trainable:
            self.freeze()

        fake_img = torch.rand(1,3,256,256) ## pass fake img to the model to get the channel size of each inception block
        self.base_model(fake_img)
        self.channel_size = [block.size()[1] for block in self.features]
        self.features = []

    def forward(self,img):
        self.base_model(img)

    def get_MLSP(self,img,feature_type,resize = True):
        self.base_model(img)
        if resize:
            print(resize)
            if feature_type == 'narrow':
                MLSP = [F.adaptive_avg_pool2d(block, (1, 1)) for block in self.features]
                for i in range(len(MLSP)):
                    MLSP[i] = MLSP[i].squeeze(2).squeeze(2)

            if feature_type == 'wide':
                MLSP = [F.interpolate(block,mode = 'bilinear', size = 7) for block in self.features]
            
            MLSP = torch.cat(MLSP,dim = 1)
            self.features = []
        else:
            MLSP = self.features
            self.features = []
        return MLSP



class head_block():
    def conv_block(self,inc,outc,ker,padding = 1,avgpool = False):
        modules = []
        modules.append(nn.Dropout(0.5))
        if avgpool:
            modules.append(AvgPool2d(3,1,1))
        modules.append(Conv2d(inc,outc,ker,padding = padding))
        modules.append(nn.BatchNorm2d(outc))
        modules.append(nn.ReLU())
        return Sequential(*modules)

    def mlsp_cnn_gap_attn(self,num_channels,attention=True):
        blocks = []
        scale = 1
        all_channels = np.sum(num_channels)
        for num_channel in num_channels:
            blocks.append(Sequential(#self.conv_block(num_channel,num_channel//scale,1,0),
                                     #self.conv_block(num_channel//scale,num_channel//scale,3,1),
                                     CBAM(num_channel//scale,reduction_ratio=16)
                          ))

        return ModuleList(blocks)


class Head(nn.Module):
    def __init__(self,head_type,num_channel):
        super(Head, self).__init__()
        if head_type == 'mlsp_cnn_gap_attn':
            self.head = getattr(head_block(),head_type)(num_channel)
        self.head_type = head_type
        self.num_ch = num_channel
        self.dense = Sequential(Linear(960,10))
    def forward(self,features):
        if self.head_type == 'mlsp_gap':
            x = torch.cat([F.adaptive_avg_pool2d(feature, (1, 1)) for feature in features],dim=1)
        else:
            x = torch.cat([F.adaptive_avg_pool2d(block(feature)+feature, (1, 1)) for feature,block in zip(features,self.head)],dim=1)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        return x

class Fmodel(nn.Module):
    def __init__(self, head_type):
        super(Fmodel,self).__init__()
        self.bmodel = Base()
        self.head = Head(head_type,self.bmodel.channel_size)
        self.feature_type = 'narrow'    
        self.resize = False
        
    def forward(self,img):
        x = self.bmodel.get_MLSP(img,self.feature_type,self.resize)
        x = self.head(x)
        return x

    def unfreeze(self):
        self.bmodel.unfreeze()
    
    def freeze(self):
        self.bmodel.freeze()




This will be the 3 model configs that you should train and generate the attacks on.  

In [ ]:
from tqdm.auto import tqdm, trange
import torchattacks

def train_model(model, dataloaders, criterion, optimizer, path, num_epochs=3,adv = False,alt=False):
    
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/225, steps=7, random_start=True)
    atk_fgsm = torchattacks.FGSM(model, eps=8/255)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']: # Each epoch has a training and validation phase
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            size = 0

            iterator = iter(dataloaders[phase])
            for _ in tqdm(range(len(iterator))): # Iterate over data
                inputs, labels = next(iterator) 
                if adv:
                    if phase=='train':
                        if alt:
                            i = np.random.choice(len(inputs),len(inputs)//2,replace=False)
                            inputs_adv = atk(inputs[i], labels[i])
                            inputs = inputs.to(device)
                            inputs[i] = inputs_adv
                        else:
                            inputs = atk(inputs,labels)

                    else:
                        if alt:
                            inputs = torch.cat((inputs.to(device), atk_fgsm(inputs, labels)),dim=0)
                            labels = torch.cat((labels,labels),dim=0)
                        else:
                            inputs = atk(inputs, labels)
                
                else:
                    inputs = inputs.to(device)

                labels = labels.to(device)
                optimizer.zero_grad() # Zero the parameter gradients

                with torch.set_grad_enabled(phase == 'train'): # Forward. Track history if only in train
                    
                    if phase == 'train': # Backward + optimize only if in training phase
  
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                            
                        _, preds = torch.max(outputs, 1)
                        loss.backward()
                        optimizer.step()
                        
                    
                    if phase == 'val':
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                size+=len(preds)

            epoch_loss = running_loss / size
            
            
            if phase == 'train': # Adjust learning rate based on val loss
                lr_scheduler.step(epoch_loss)
                
            epoch_acc = running_corrects.double() / size
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc  >best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(),path)
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


Evaluation (pre-attack)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(3,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(-1,64)
        out = self.fc_layer(out)

        return out

In [ ]:
# model = _resnet('resnet', BasicBlock, [1, 1, 1, 1], False, True)
model = models.resnet34(pretrained=False)
model.fc = Linear(512,10)
# model = Fmodel('mlsp_gap')
# model = CNN()

model(torch.rand((2,3,224,224)))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001,     momentum=0.9, weight_decay= 5e-4)
# lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=5, verbose=True, min_lr =1e-6)

optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=1, verbose=True, min_lr =1e-6)
# lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

In [ ]:
path = '/content/gdrive/MyDrive/cifar10/saved_model/resnet34_base.pt'
epochs = 100 #50
model, _ = train_model(model, {"train": train_loader, "val": val_loader}, criterion, optimizer, path, epochs,False,False)

Epoch 0/99
----------


  0%|          | 0/391 [00:00<?, ?it/s]

train Loss: 1.7001 Acc: 0.3840


  0%|          | 0/50 [00:00<?, ?it/s]

val Loss: 1.4442 Acc: 0.4840

Epoch 1/99
----------


...

train Loss: 0.0246 Acc: 0.9921


  0%|          | 0/50 [00:00<?, ?it/s]

val Loss: 2.0019 Acc: 0.6312

Epoch 32/99
----------


  0%|          | 0/391 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
correct = 0 
size = 0
model.eval().cuda()

with  torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Make predictions.
        prediction= model(inputs)

        # Retrieve predictions indexes.
        _, predicted_class = torch.max(prediction.data, 1)

        # Compute number of correct predictions.
        correct += (predicted_class == labels).float().sum().item()
        size+=len(prediction)
test_accuracy = correct / size
print('Test accuracy: {}'.format(test_accuracy))

Test accuracy: 0.6306


3 types of white box attack 

In [ ]:

model.eval()
atks = [
    torchattacks.FGSM(model, eps=8/255),
    torchattacks.PGD(model, eps=8/255, alpha=2/225, steps=7, random_start=True),
]
for i in [0,4]:
    print(i)
    correct = 0
    start = time.time()
    size = 0
    for images, labels in test_loader:   
        adv_images = atks[i](images, labels)
        labels = labels.to(device)
        outputs = model(adv_images)

        _, pre = torch.max(outputs.data, 1)
        correct += (pre == labels).float().sum().item()
        size+=len(labels)

    # print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Robust accuracy: %.2f ' % (correct / size))


0
Robust accuracy: 0.23 
4
Robust accuracy: 0.05 


In [ ]:
#FGSM attack 

#def fgsm_attack(model, loss, images, labels, eps) :
    
 #   images = images.to(device)
#    labels = labels.to(device)
#    images.requires_grad = True
            
#    outputs = model(images)
    
 #   model.zero_grad()
 #   cost = loss(outputs, labels).to(device)
  #  cost.backward()
    
   # attack_images = images + eps*images.grad.sign()
  #  attack_images = torch.clamp(attack_images, 0, 1)
    
    #return attack_images

In [ ]:
torch.save(model.state_dict(),'/content/gdrive/MyDrive/imagenet/saved_model/resnet_attack_adverserial.pt' )


In [ ]:
#torch.save(model.state_dict(),'/content/gdrive/MyDrive/imagenet/saved_model/Fmodel_mlsp_adverserial.pt' )

In [ ]:
#torch.save(model.state_dict(),'/content/gdrive/MyDrive/imagenet/saved_model/Fmodel_mlsp_cnn_gap_attn.pt' )

In [ ]:
#model = Fmodel('mlsp_gap')
model = models.resnet18(pretrained=False)
model.to(device)
#model = torch.load('/content/gdrive/MyDrive/imagenet/saved_model/atk/resnet18_base.pt')
model.load_state_dict(torch.load('/content/gdrive/MyDrive/imagenet/saved_model/attk/resnet18_base.pt'))
#model.load_state_dict(torch.load('/content/gdrive/MyDrive/imagenet/saved_model/resnet_attack_adverserial.pt'))

<All keys matched successfully>

In [ ]:
!ls /content/gdrive/MyDrive/imagenet/saved_model/attk

attk				  Fmodel_mlsp_adverserial.pt
best_resnet18.pt		  Fmodel_mlsp_cnn_gap_attn.pt
custom_attention.pt		  InceptionV3.pt
custom_attn_multi_12M_noskip.pt   InceptionV3true.pt
custom_attn_multi_18M_no_skip.pt  latest_baseline_vgg16.pt
custom_attn_multi_18M.pt	  latest_resnet18_custom_attention.pt
custom_attn_multi.pt		  mlsp_gap_attack.pt
custom_attn_skip_16M.pt		  multi_3FC.pt
custom_no_attn_multi_18M_skip.pt  pool_3FC_5x5.pt
custom_noattn_noskip_16M.pt	  pool_3FC.pt
custom_no_attn_skip_16M.pt	  resnet18_adv_train.pt
custom_only_att_3convfinal.pt	  resnet18_final_custom_attention.pt
custom_only_att.pt		  resnet34_attack.pt
custom.pt			  resnet_attack_adverserial.pt
custom_skip.pt			  resnet_attack.pt
custom_stepwise.pt
